In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys; sys.path.insert(0, '..')

In [3]:
import fiona
import geopandas as gpd
from pathlib import Path
import numpy as np
import osmnx as ox
from shapely.geometry import LineString, Point
from tqdm import tqdm

In [4]:
from main import prepare_data_for_place, OUTPUT_COLUMNS
from src.route import get_route_gdf

In [5]:
def is_connected(g):
    import networkx as nx
    return nx.is_connected(g.to_undirected())

## Prepare out dir

In [6]:
import os
import shutil

OUT_PATH = Path("../data/out/notebook/")

# Delete the directory if it exists
if OUT_PATH.exists():
    shutil.rmtree(OUT_PATH)

# Recreate the directory
OUT_PATH.mkdir(parents=True, exist_ok=True)

## Load bike network

In [7]:
nodes, edges = prepare_data_for_place("Somerville, MA, USA")

> Getting bike network for Somerville, MA, USA
> Processing network for Somerville, MA, USA
> MODEL 1: Preparing speed data for Somerville, MA, USA
> MODEL 2: Preparing separation level data for Somerville, MA, USA
> MODEL 3: Preparing street category data for Somerville, MA, USA
> MODEL 4: Preparing lanes data for Somerville, MA, USA
> MODEL: Preparing composite score for Somerville, MA, USA


In [8]:
edges = edges[OUTPUT_COLUMNS]

In [9]:
G = ox.graph_from_gdfs(nodes, edges)

In [10]:
edges.sample(3)

,,,name,maxspeed_0,maxspeed_int,maxspeed_int_score,separation_level,separation_level_score,street_0,street_classification,street_classification_score,composite_score,width_float,width_half,geometry
u,v,key,,,,,,,,,,,,,
1057,153,0,Linden Avenue,20 mph,20.0,0.0,none,4.0,residential,residential,2,2.8,13.7,6.85,"LINESTRING (-71.11393 42.38805, -71.11427 42.3..."
1185,435,0,Somerville Community Path,NaN,NaN,NaN,separate,0.0,cycleway,dedicated_path,0,0.0,3.048,1.524,"LINESTRING (-71.10408 42.39124, -71.10407 42.3..."
391,392,0,Maxwells Green,NaN,NaN,NaN,none,4.0,service,residential,2,3.5,10.0,5.0,"LINESTRING (-71.10799 42.39425, -71.10777 42.3..."


In [11]:
is_connected(G)

True

## Load Schools

In [12]:
school_gdb_path = "../data/raw/SafeRoutesGISLayers.gdb.zip"

In [13]:
layers = fiona.listlayers(school_gdb_path)
layers

['SafetyZoneStreets', 'PublicSchools', 'Sidewalks', 'Signalized_Intersections']

In [16]:
# read school data
schools_gdf = gpd.read_file(school_gdb_path, layer='PublicSchools')

# save schools polygons
schools_gdf.to_file((OUT_PATH / "schools_poly.gpkg"), driver="GPKG")

# make geom col into centroids
schools_gdf['geometry'] = schools_gdf.centroid

# save schools polygons
schools_gdf.to_file((OUT_PATH / "schools_centroid.gpkg"), driver="GPKG")

In [17]:
schools_gdf.head(3)

,Name,GlobalID,Shape_Length,Shape_Area,geometry
0,West Somerville Neighborhood School,{423648E4-357B-4C51-8323-18DE5B5EF869},857.126130,20546.222891,POINT (757029.484 2973287.291)
1,Brown School,{32ED129B-38AE-4E8F-A71B-A18126973D75},511.378543,10156.639765,POINT (760400.444 2970061.762)
2,Healey School,{374CFA80-E38D-4411-AB46-7868E8DA8468},900.132189,38897.272280,POINT (765459.28 2970148.61)


## Load census blocks

In [18]:
# read census blocks
census_blocks = gpd.read_file("../data/raw/Census_2020_Blocks.zip")

# filter by TOWN attribute
somerville_census_blocks = census_blocks[census_blocks['TOWN'] == "SOMERVILLE"].copy()

# reset index
somerville_census_blocks = somerville_census_blocks.reset_index(drop=True)

# get a sample (half)
half_n_census_blocks = len(somerville_census_blocks) // 2
somerville_census_sample = somerville_census_blocks.sample(half_n_census_blocks)

# save polygon version
somerville_census_blocks.to_file((OUT_PATH / "somer_blocks_poly.gpkg"), driver="GPKG")
somerville_census_sample.to_file((OUT_PATH / "somer_sample_poly.gpkg"), driver="GPKG")

# convert geometry to centroid
somerville_census_blocks['geometry'] = somerville_census_blocks.centroid
somerville_census_sample['geometry'] = somerville_census_sample.centroid

# save centroid version
somerville_census_blocks.to_file((OUT_PATH / "somer_blocks_centroid.gpkg"), driver="GPKG")
somerville_census_sample.to_file((OUT_PATH / "somer_sample_centroid.gpkg"), driver="GPKG")

In [19]:
somerville_census_blocks.head(3)

,OBJECTID,STATEFP20,COUNTYFP20,TRACTCE20,BLOCKCE20,GEOID20,NAME20,MTFCC20,ALAND20,AWATER20,...,AREA_SQFT,AREA_ACRES,TOWN,TOWN_ID,BLKGRP20,TRACT20,COUSUBFP,SHAPEAREA,SHAPELEN,geometry
0,51243,25,017,351002,2002,250173510022002,Block 2002,G5040,18175,0,...,195621.53,4.49,SOMERVILLE,274,250173510022,25017351002,62535,18173.907063,626.570381,POINT (231829.87 904798.558)
1,51295,25,017,350400,2007,250173504002007,Block 2007,G5040,17571,0,...,189123.23,4.34,SOMERVILLE,274,250173504002,25017350400,62535,17570.193552,675.342228,POINT (231953.871 905367.479)
2,51456,25,017,350108,2003,250173501082003,Block 2003,G5040,19544,0,...,210356.28,4.83,SOMERVILLE,274,250173501082,25017350108,62535,19542.816299,707.710510,POINT (233007.237 905138.728)


## Make sure everything has same crs

- EPSG:26986 =  NAD83 / Massachusetts Mainland Meters
- EPSG:4326 = WGS 84 / web

In [20]:
def crs_first_line(gdf):
    return str(gdf.crs).splitlines()[0]

In [21]:
print("somerville_census_blocks:", crs_first_line(somerville_census_blocks))
print("schools_gdf             :", crs_first_line(schools_gdf))
print("edges                   :", crs_first_line(edges))
print("nodes                   :", crs_first_line(nodes))

somerville_census_blocks: EPSG:26986
schools_gdf             : EPSG:6492
edges                   : EPSG:4326
nodes                   : EPSG:4326


In [22]:
# use this one
use_crs = edges.crs
# use_crs = somerville_census_blocks.crs
# use_crs = projected_crs

# make them match
somerville_census_blocks = somerville_census_blocks.to_crs(use_crs)
schools_gdf = schools_gdf.to_crs(use_crs)
edges = edges.to_crs(use_crs)

In [23]:
print("somerville_census_blocks:", crs_first_line(somerville_census_blocks))
print("schools_gdf             :", crs_first_line(schools_gdf))
print("edges                   :", crs_first_line(edges))
print("nodes                   :", crs_first_line(nodes))

somerville_census_blocks: EPSG:4326
schools_gdf             : EPSG:4326
edges                   : EPSG:4326
nodes                   : EPSG:4326


## Routing

In [24]:
G = ox.graph_from_gdfs(nodes, edges)

In [28]:
# pick a school
dest_point = schools_gdf.loc[0, 'geometry']

In [29]:
# pick a census centroid
orig_point = somerville_census_sample.loc[0, 'geometry']

In [30]:
route_gdf = get_route_gdf(G, orig_point, dest_point)

In [39]:
route_gdf.to_file("../data/out/notebook/route.gpkg", driver="GPKG")

In [46]:
route_gdf

,type,geometry,composite_score
0,start,POINT (232842.61947 902861.73213),NaN
1,end,POINT (-71.12654 42.40625),NaN
2,route,"LINESTRING (-71.10134 42.37447, -71.10178 42.3...",1.352612


## Route loop

In [56]:
# pick a school
school = schools_gdf.iloc[0]
dest_point = schools_gdf.loc[0, 'geometry']

In [67]:
for i, row in tqdm(somerville_census_sample.iterrows(), total=len(somerville_census_sample)):
    orig_point = row['geometry']
    dest_point = school['geometry']
    try:
        route_gdf = get_route_gdf(G, orig_point, dest_point)
    except Exception as e:
        print(f"Error on index {i}: {e}")
        route_gdf = gpd.GeoDataFrame()

    if not route_gdf.empty:
        route_gdf["school_name"] = school["Name"]

 19%|████████████▏                                                     | 65/351 [00:00<00:02, 107.77it/s]

Error on index 543: No route found between start and end


100%|█████████████████████████████████████████████████████████████████| 351/351 [00:03<00:00, 107.95it/s]

Error on index 438: No route found between start and end


In [68]:
route_gdf

,type,geometry,composite_score,school_name
0,start,POINT (232842.61947 902861.73213),NaN,West Somerville Neighborhood School
1,end,POINT (-71.12654 42.40625),NaN,West Somerville Neighborhood School
2,route,"LINESTRING (-71.10134 42.37447, -71.10178 42.3...",1.352612,West Somerville Neighborhood School


Name                      West Somerville Neighborhood School
GlobalID               {423648E4-357B-4C51-8323-18DE5B5EF869}
Shape_Length                                        857.12613
Shape_Area                                       20546.222891
geometry        POINT (-71.12653785659657 42.406246162695865)
Name: 0, dtype: object